In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('all_train.csv')

FileNotFoundError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/all_train.csv')

In [4]:
data.head()

,# label,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f21,f22,f23,f24,f25,f26,mass
0,1.0,-0.346368,0.416306,0.999236,0.475342,0.427493,-0.005984,1.989833,0.344530,1.566297,...,4.105282,0.267826,0.378718,1.743123,3.406367,4.350537,-0.352571,1.130032,2.227706,1000.0
1,1.0,1.708236,-0.319394,-1.241873,-0.887231,-0.871906,-0.005984,-0.001047,-1.038225,0.655748,...,-1.178141,-0.877361,-1.483769,-0.573682,-1.693781,-0.545062,-0.299118,-0.662942,-0.193019,750.0
2,0.0,-0.360693,1.794174,0.264738,-0.472273,-0.292344,-1.054221,-1.150495,1.423404,1.270098,...,-1.199511,0.539020,-1.590629,-0.573682,-0.543636,-0.937456,-0.300344,-0.523262,-1.506304,750.0
3,1.0,-0.377914,-0.103932,-0.649434,-2.125015,-1.643797,-0.005984,1.011112,-1.040340,-0.541991,...,0.463763,-0.006583,1.089122,-0.573682,-0.276348,-0.409272,-0.349926,-0.307123,0.529698,1250.0
4,0.0,-0.067436,-0.636762,-0.620166,-0.062551,1.588715,-0.005984,-0.595304,-1.238987,0.336844,...,-0.552837,-1.418494,-0.562982,1.743123,0.881802,0.002516,1.560950,-0.150760,-1.023889,750.0


In [5]:
data.shape

(7000000, 29)

Missing Values

In [8]:
data.isna().sum().sum()

0

Duplicates

In [9]:
data.duplicated().sum()

0

Single Values

In [10]:
single_vals = data.columns[data.eq(data.iloc[0]).all()].tolist()
print(f"There are {len(single_vals)} columns with single values: {single_vals}")

There are 0 columns with single values: []


Data Types

In [11]:
data.dtypes

# label    float64
f0         float64
f1         float64
f2         float64
f3         float64
f4         float64
f5         float64
f6         float64
f7         float64
f8         float64
f9         float64
f10        float64
f11        float64
f12        float64
f13        float64
f14        float64
f15        float64
f16        float64
f17        float64
f18        float64
f19        float64
f20        float64
f21        float64
f22        float64
f23        float64
f24        float64
f25        float64
f26        float64
mass       float64
dtype: object

In [12]:
# even distribution
data.iloc[:,0].value_counts()

1.0    3500879
0.0    3499121
Name: # label, dtype: int64

In [7]:
df = data.copy()

In [8]:
# change target variable to int32
df.iloc[:,0] = df.iloc[:,0].astype('int32')

Train Test Split

In [9]:
# change target variable name
df.rename({'# label': 'label'}, axis='columns', inplace=True)

In [10]:
y = df['label']
X = df.drop(columns = ['label'])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

Random Forest Benchmark

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
clf = RandomForestClassifier(n_estimators=20, random_state=9)
clf.fit(X_train,y_train)
preds = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, preds)}')
rec = recall_score(y_test,preds)
prec = precision_score(y_test, preds)
print(f'Recall Score: {rec}')
print(f'Precision Score: {prec}')

Accuracy: 0.8610114285714285
Recall Score: 0.8644461149378724
Precision Score: 0.858905948164557


In [ ]:
# save benchmark RF, took 28 min to run
import joblib
joblib.dump(clf,"benchmark_RF.joblib", compress=3)

# run this to load
#clf = joblib.load('benchmark_RF.joblib')

['benchmark_RF.joblib']

Neural Networks

In [12]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X_train)

In [13]:
my_model = tf.keras.Sequential()

In [14]:
scaled_X.shape

(5600000, 28)

In [15]:
my_model.add(tf.keras.Input(shape=(28,))) #input layer
my_model.add(tf.keras.layers.Dense(100, activation='sigmoid')) #dense
my_model.add(tf.keras.layers.Dense(50, activation='sigmoid')) #dense
my_model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [16]:
my_model.compile(optimizer = 'sgd', loss='BinaryCrossentropy', metrics = ['accuracy'])

In [17]:
my_model.fit(scaled_X, y_train, epochs = 5, batch_size=15)

Epoch 1/5
373334/373334 [==============================] - 1012s 3ms/step - loss: 0.3261 - accuracy: 0.8465
Epoch 2/5
373334/373334 [==============================] - 1003s 3ms/step - loss: 0.2936 - accuracy: 0.8622
Epoch 3/5
373334/373334 [==============================] - 998s 3ms/step - loss: 0.2865 - accuracy: 0.8667
Epoch 4/5
373334/373334 [==============================] - 990s 3ms/step - loss: 0.2833 - accuracy: 0.8686
Epoch 5/5
373334/373334 [==============================] - 1038s 3ms/step - loss: 0.2813 - accuracy: 0.8699


In [19]:
my_model.save('tf_model_1.h5')

In [23]:
# load model
# my_model = tf.keras.models.load_model('tf_model_1.h5')

In [20]:
scaled_Xtest = scaler.transform(X_test)

In [22]:
loss, acc = my_model.evaluate(scaled_Xtest, y_test, verbose=0)
print(f'Accuracy: {acc*100}')

Accuracy: 87.01678514480591
